## Notebook Topic: Advanced Cleaning Techniques

<ins>Learning Objectives</ins>

1. To read in data from an excel file
2. To clean messy data into a desired format

In this notebook, we'll upload and clean a dataset obtained on the [World Bank](https://databank.worldbank.org/source/gender-statistics) Database.  Since it is real world data, there will inevitably be missing data or unnessary information we need to filter out.  We will use what we learned in Chapters 5 and 6 about **pandas** to be helpful in this process, but we may also need skills in Chapter 7 <span style="color:purple">(so I recommend you either read it before you begin this notebook, or immediately after you finish)</span>.

**Section I: Loading Data from an Excel File**

In previous notebooks, we either made up data (e.g., using ```numpy.random.normal()```) or we used pre-cleaned data (e.g., using the **pydataset** library).  Generally, 80% of the life of a data scientist is spent collecting data and then cleaning it.  I've done this first part for you using a well known open data site, World Bank Group.  We need only load it into our <ins>workspace</ins> in order to start cleaning it.

In [28]:
import pandas as pd

# pandas has a nice read_excel function!  There are many others listed in Ch 6
# insert the file path/name
# specify sheet name and engine to be used
DF = pd.read_excel("P_Data_Extract_From_Gender_Statistics.xlsx", sheet_name = "Data", engine="openpyxl")

Each file will require different parameters, but for the purposes of this notebook the line above is how we load the data into Jupyter.  <span style="color:purple">Go ahead and explore the data a bit below to understand it better</span>.

**Section II: Understanding Our Data**

I personally noticed that there are different countries listed, and I wonder whether each row is a different country or if there are $n < N$ different countries and we can group the rows by their identifying country.  So I'll check:

In [ ]:
DF.___ # fill in the blank with the function that gives the number of rows and columns

In [ ]:
DF["Country Name"].unique().size # looks at the column of country names, selects the unique country names, and then counts how many

Ah ha!  Before we deal with that, let's remove the columns "Series Code" and "Country Code" since they don't currently add anything to our examination.

In [29]:
copy_DF = DF.copy()

In [35]:
DF = DF.drop(labels = ["Series Code", "Country Code"], axis = 1)

Now we'll group the data by country.

In [37]:
DF = DF.sort_values("Country Name")

With data grouped by country, we can do a country-to-country comparison based on the variables.  <span style="color:purple">But what are those variables?  List what you see from the next line of code.</span>

In [ ]:
DF.iloc[:,0]

We can do better than this.  <span style="color:purple">What do you observe between the results of the previous code chunk and the results of the code chunk below?</span>

In [ ]:
DF.iloc[:,0].dropna() # removes the missing data since it's meaningless

But we still don't yet know what are the unique list of variables and how many there are.  

<span style="color:purple">How many unique categorical variables are there?  Modify the below line of code to get the list of unique variables and to save it a Python variable called ```list_of_categ_variables```.  </span>

In [ ]:
DF.iloc[:,0].dropna().unique().size

Now that we know a bit more about the information given, we can see that maybe our country-to-country comparison may be on "The Independence of Women".

**Section III: Reformatting Data**

I personally don't like the stacked layout because it's difficult for me to see what's going on.  So, I'm going to create a different data frame with the rows being each individual country and each column being each individual categorical variable and each element being the value from ```DF```'s column called ```2022 [YR2022]```.

In [ ]:
new_tab = pd.pivot_table(DF, values = ['2022 [YR2022]'], index = 'Country Name', columns = 'Series Name', aggfunc='first')
new_tab.columns = new_tab.columns.droplevel(0)

# print out new_tab (if you don't remember how, look at notebooks 00-06) and see what it looks like

From the reshaping of the data, we can actually see that much of the data is missing!  And it doesn't show up as ```NaN``` or ```NA```, which we filtered out earlier!  Tricky.  This missing data is ```..``` and is likely identified as a string object type.

<span style="color:purple">Remind me what string objects are.</span>

Two of the countries seem to have absolutely no data available.  <span style="color:purple">Which are they?

Let's check to see if it's true!

In [ ]:
DF[DF['Country Name'] == INSERTNAMEHERE]

So perhaps we can remove that data completely from our table.

In [ ]:
removed_rows_DF = DF.loc[DF['Country Name'] != 'Gibraltar']
removed_rows_DF = removed_rows_DF.loc[removed_rows_DF['Country Name'] != 'Guam']
new_tab = pd.pivot_table(removed_rows_DF, values = ['2022 [YR2022]'], index = 'Country Name', columns = 'Series Name', aggfunc='first')
new_tab.columns = new_tab.columns.droplevel(0)

# print out new_tab (if you don't remember how, look at notebooks 00-06) and see what it looks like

We also make note of some columns that are completely useless!  For example, *Account, female (% age 15+)* and *Account, male (% age 15+)* have only ```..```.  We can infer that these variables are "% of females/males who are over 15+ who have [bank] accounts" and that there is no data for these variables for the selected countries.

So, we drop those variables from the original data frame and then we update the table again.

In [ ]:
indx = removed_rows_DF['Series Name'] == 'Account, female (% age 15+)' # gets Boolean information
indx = removed_rows_DF[indx].index # gets row numbers
more_removed_rows_DF = removed_rows_DF.drop(indx) # drops the rows associated with those row numbers

<span style="color:purple">Duplicate this process for 'Account, male (% age 15+)' in a code chunk below.</span>

Now we'll update our table!

In [ ]:
new_tab = pd.pivot_table(more_removed_rows_DF, values = ['2022 [YR2022]'], index = 'Country Name', columns = 'Series Name', aggfunc='first')
new_tab.columns = new_tab.columns.droplevel(0)

# print out new_tab (if you don't remember how, look at notebooks 00-06) and see what it looks like

There are still a lot of columns that seem pointless for us.  There must be a faster way, ugh.  

Notice that the blank data seems to start around column #9 and ends around column #39. <span style="color:purple">Check the output of the following lines in a code chunk.</span>

```
   new_tab.iloc[:,7] # this is column 8
   new_tab.iloc[:,8] # this is column 9
   new_tab.iloc[:,20] # this is column 21; we should check something in between before removing it completely
   new_tab.iloc[:,37] # this is column 38
   new_tab.iloc[:,38] # this is column 39
```

So, perhaps we can just drop the columns with index 8 through 37 and get this over with.  Let's see!

In [157]:
new_tab.drop(new_tab.iloc[:,8:37], axis = 1, inplace = True)

In [ ]:
new_tab

woot!  We now have a cleaner dataset.  What a workout!

**Section IV: Writing Our New Dataset to an Excel File**

Sometimes, after we've spent so much time cleaning our data, we aren't yet ready to do analysis... it was a long day, man.  We don't want to clean it again.  We simply want to work with the newer, nicer data, but we want to do it tomorrow.  So... we save it until next time.

In [159]:
new_tab.to_excel("data_for_tomorrow.xlsx", index = True, header = True)

## Conclusion

The way we clean one dataset is going to be significantly different than the way we clean another.  There is no cookie cutter process, so you should definitely look at all your resources on the Google Classroom when cleaning a dataset!  Usually you can type into Google Search exactly what you want to do and a forum (my favorite is [stackoverflow](https://stackoverflow.com/questions/)) will have a discussion on how to do what you want, or something similar enough to gain from the discussion.

I still recommend reading Chapters 7 and 8 from our textbook for useful tips on cleaning data.

